In [ ]:
# ignore warnings

import warnings
warnings.filterwarnings("ignore")

Installing Dependencies

In [ ]:
! pip install torch==1.8.2+cu111 torchvision==0.9.2+cu111 torchaudio===0.8.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
! pip install datasets
! pip install rouge

Looking in links: https://download.pytorch.org/whl/lts/1.8/torch_lts.html
ERROR: Could not find a version that satisfies the requirement torch==1.8.2+cu111 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0)
ERROR: No matching distribution found for torch==1.8.2+cu111
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 r

Load dataset

In [ ]:
!pip install datasets
from datasets import load_dataset
multi_news = load_dataset("multi_news", split="test")

Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

View Data

In [ ]:
multi_news.to_pandas()

,document,summary
0,GOP Eyes Gains As Voters In 11 States Pick Gov...,– It's a race for the governor's mansion in 11...
1,\n \n \n \n UPDATE: 4/19/2001 Read Richard Met...,– It turns out Facebook is only guilty of abou...
2,It's the Golden State's latest version of the ...,– Not a big fan of Southern California? Neithe...
3,The seed for this crawl was a list of every ho...,– Why did Microsoft buy Nokia's phone business...
4,After a year in which liberals scored impressi...,– The Supreme Court is facing a docket of high...
...,...,...
5617,Tweet with a location \n \n You can add locati...,– The traditional end-of-summit group photo at...
5618,Loic Venance/AFP/Getty Images \n \n The awards...,– Sofia Coppola scored a historic victory at t...
5619,(CNN) A federal criminal investigation into a ...,– The duck boat sinking that killed 17 on a Mi...
5620,An archive of the public statements deleted by...,– Note to tweeting politicians: Watch what you...


Train test split (80:20 ratio respectively)

In [ ]:
multi_news = multi_news.train_test_split(test_size=0.2)

Tokenizing Training and testing sets

In [ ]:
! pip install transformers

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    labels = tokenizer(text=examples["summary"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_multi_news = multi_news.map(preprocess_function, batched=True)

Map:   0%|          | 0/4497 [00:00<?, ? examples/s]

Map:   0%|          | 0/1125 [00:00<?, ? examples/s]

Model Loading

In [ ]:
from transformers import DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model='t5-small')
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Defining Hyperparameters

In [ ]:
!pip install accelerate -U
!pip install transformers[torch]
import accelerate
import transformers

from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    weight_decay=0.01,
    save_total_limit=3,  # This should likely be an integer, not a float, representing the number of checkpoint files to keep.
    num_train_epochs=10,
    fp16=True
)




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 1.3 MB/s eta 0:00:00


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

Initializing Trainer

Train the Text Summarization Model

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from datasets import load_dataset

# Assuming the rest of your setup is correct and already provided

# Load the dataset
dataset = load_dataset("multi_news", split={'train': 'train', 'test': 'test'})

# Update the tokenize function to include both the inputs and targets
def tokenize_function(examples):
    # Tokenize the inputs and targets (you may need to adjust field names based on your dataset)
    model_inputs = tokenizer(examples["document"], padding="max_length", truncation=True, max_length=512)

    # Assume "summary" is the field for the target summaries
    # Note: T5 expects the target sequence to be prefixed with "summarize: " for summarization tasks
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], padding="max_length", truncation=True, max_length=128)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_multi_news = dataset.map(tokenize_function, batched=True, remove_columns=dataset["train"].column_names)

# Define the data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding=True, return_tensors="pt")

# Now, creating the Seq2SeqTrainer with the defined model and other components
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_multi_news["train"],
    eval_dataset=tokenized_multi_news["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Train the model
trainer.train()


Map:   0%|          | 0/44972 [00:00<?, ? examples/s]

Map:   0%|          | 0/5622 [00:00<?, ? examples/s]

NameError: name 'training_args' is not defined

Evaluate model on a single document

In [ ]:
document = "National Archives Yes, it's that time again, folks. It's the first Friday of the month, when for one ever-so-brief moment the interests of Wall Street, Washington and Main Street are all aligned on one thing: Jobs. A fresh update on the U.S. employment situation for January hits the wires at 8:30 a.m. New York time offering one of the most important snapshots on how the economy fared during the previous month. Expectations are for 203,000 new jobs to be created, according to economists polled by Dow Jones Newswires, compared to 227,000 jobs added in February. The unemployment rate is expected to hold steady at 8.3%. Here at MarketBeat HQ, we’ll be offering color commentary before and after the data crosses the wires. Feel free to weigh-in yourself, via the comments section. And while you're here, why don't you sign up to follow us on Twitter. Enjoy the show. ||||| Employers pulled back sharply on hiring last month, a reminder that the U.S. economy may not be growing fast enough to sustain robust job growth. The unemployment rate dipped, but mostly because more Americans stopped looking for work. The Labor Department says the economy added 120,000 jobs in March, down from more than 200,000 in each of the previous three months. The unemployment rate fell to 8.2 percent, the lowest since January 2009. The rate dropped because fewer people searched for jobs. The official unemployment tally only includes those seeking work. The economy has added 858,000 jobs since December _ the best four months of hiring in two years. But Federal Reserve Chairman Ben Bernanke has cautioned that the current hiring pace is unlikely to continue without more consumer spending."
human_summary = """– The unemployment rate dropped to 8.2% last month, but the economy only added 120,000 jobs, when 203,000 new jobs had been predicted, according to today's jobs report. Reaction on the Wall Street Journal's MarketBeat Blog was swift: "Woah!!! Bad number." The unemployment rate, however, is better news; it had been expected to hold steady at 8.3%. But the AP notes that the dip is mostly due to more Americans giving up on seeking employment."""

In [ ]:
def predict_summary(document):
  device = model.device
  tokenized = tokenizer([document], truncation =True, padding ='longest',return_tensors='pt')
  tokenized = {k: v.to(device) for k, v in tokenized.items()}
  tokenized_result = model.generate(**tokenized, max_length=128)
  tokenized_result = tokenized_result.to('cpu')
  predicted_summary = tokenizer.decode(tokenized_result[0])
  return predicted_summary

In [ ]:
human_summary

In [ ]:
predicted_summary = predict_summary(document)
predicted_summary

Evaluate using Rouge Scores

ROUGE (Recall-Oriented Understudy for Gisting Evaluation) is a suite of metrics for evaluating the quality of text summaries. It compares a set of reference summaries with a set of generated summaries, and computes a score based on the overlap between the two.

In [ ]:
from rouge import Rouge

def get_rouge_scores(actual_summary, predicted_summary):
    rouge = Rouge()
    scores = rouge.get_scores(predicted_summary, actual_summary)
    return [scores[0]['rouge-1']['f'], scores[0]['rouge-2']['f'], scores[0]['rouge-l']['f']]

In [ ]:
# import validation dataset

import pandas as pd
val_data = pd.read_excel('/content/Validation_Data.xlsx')
val_data

In [ ]:
# rouge score of validation data

from tqdm import tqdm

rouge1_scores = []
rouge2_scores = []
rougel_scores = []

pred_summary_list = []

for i in tqdm(range(len(val_data))):

  doc = val_data.loc[i]['document']
  pred_summary = predict_summary(document)
  human_summary = val_data.loc[i]['summary']

  score = get_rouge_scores(human_summary, pred_summary)

  rouge1_scores.append(score[0])
  rouge2_scores.append(score[1])
  rougel_scores.append(score[2])

  pred_summary_list.append(pred_summary)

val_data["pred_summary"] = pred_summary_list

val_data['rouge1'] = rouge1_scores
val_data['rouge2'] = rouge2_scores
val_data['rougel'] = rougel_scores

val_data

In [ ]:
# average rouge 1
val_data['rouge1'].mean()

In [ ]:
# average rouge 2
val_data['rouge2'].mean()

In [ ]:
# average rouge l
val_data['rougel'].mean()